# html 파일 정제 후 사용가능한 데이터 프레임으로 제작

In [1]:
import numpy as np
import pandas as pd

In [2]:
# html 파일은 lxml이라는 패키지가 필요함 -> python3의 경우 pip3로 다운받아야 에러가 안뜸
#!pip3 install lxml

In [2]:
# 결과물 데이터 프레임
result = pd.DataFrame()
# 년도 설정
years = np.arange(2010, 2020)

In [3]:
for year in years:
    # 파일 불러오기
    df = pd.read_html(f"./boxofficeFiles/boxoffice_{year}.html", encoding="utf-8")
    df = df[1]

    # na값 제외하기
    for col in df.columns:
        df = df.drop(df[df[col].isna()].index)
    print(df.isna().sum().sum())
    df.reset_index(drop=True, inplace=True)

    # 년도별 박스오피스에는 해당 년도의 영화를 제외한 영화들도 포함되어 있음(재개봉 혹은 기타 사유로) 이는 제외
    # 전년도 12월 ~ 다음년도 1월 사이의 개봉일을 제외하고 나머지 개봉일들은 삭제하기
    drop_index = []
    for i in range(len(df)):
        if ((int(df.loc[i,"개봉일"].split("-")[0]) > (year + 1)) | (int(df.loc[i,"개봉일"].split("-")[0]) < (year - 1))) | (
            ((int(df.loc[i,"개봉일"].split("-")[0]) == (year - 1)) & (int(df.loc[i,"개봉일"].split("-")[1]) < 12)) | \
                ((int(df.loc[i,"개봉일"].split("-")[0]) == (year + 1)) & (int(df.loc[i,"개봉일"].split("-")[1]) > 1))
        ):
            drop_index.append(i)
    df.drop(drop_index, inplace=True)
    df.reset_index(drop=True, inplace=True)

    # 최종 데이터셋에 합치기
    result = pd.concat([result, df])
    print(f"{year} clear")

0
2010 clear
0
2011 clear
0
2012 clear
0
2013 clear
0
2014 clear
0
2015 clear
0
2016 clear
0
2017 clear
0
2018 clear
0
2019 clear


In [4]:
# 중복되거나 사용가치가 없는 Columns 제거
result.drop(["순위", "매출액 점유율", "상영횟수", "대표국적", "국적"], axis=1, inplace = True)

In [6]:
# 관객수에 따라 정렬
result.sort_values("관객수", ascending=False, inplace=True)

In [7]:
# 결과파일로 저장하기
result.to_csv("./boxofficeFiles/boxoffice_base.csv", index=False)